In [1]:
# 0    age_group
# 2    retailer
# 5    retailer_segment
# 6    retailer_site
# 7    retailer_type
# 8    sales_demographic
# 9    sales_territory

# 1    order_details
# 2    order_header
# 3    order_method
# 4    product
# 5    product_line
# 6    product_type
# 7    retailer_site
# 10    sales_branch
# 11    sales_staff
# 12    SALES_TARGETData

# 1    sales_branch

In [2]:
import sqlite3 as sql
import warnings as warn

import pandas as pd

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import LabelEncoder

In [3]:
warn.filterwarnings("ignore")

In [4]:
conn1 = sql.connect("Great_Outdoors_Data_SQLite/go_crm.sqlite")
conn2 = sql.connect("Great_Outdoors_Data_SQLite/go_sales.sqlite")
conn3 = sql.connect("Great_Outdoors_Data_SQLite/go_staff.sqlite")

# Iterate over all tables from conn1
for i, conn in enumerate([conn1, conn2, conn3]):
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

    for table in tables:
        globals()[table[0] + str(i)] = pd.read_sql_query(f"SELECT * FROM {table[0]}", conn)
        print(f"{table[0]}: {globals()[table[0] + str(i)].shape}")
        print(globals()[table[0] + str(i)])

age_group: (6, 4)
  AGE_GROUP_CODE UPPER_AGE LOWER_AGE TRIAL219
0              1        20         0        T
1              2        30        21        T
2              3        40        31        T
3              4        50        41        T
4              5        60        51        T
5              6        70        61        T
country: (21, 5)
   COUNTRY_CODE      COUNTRY_EN FLAG_IMAGE SALES_TERRITORY_CODE TRIAL219
0             1          France        F01                    6        T
1            10          Sweden        F10                    5        T
2            11           Japan        F11                    4        T
3            12          Taiwan        F12                    4        T
4            13           Korea        F13                    4        T
5            14           China        F14                    4        T
6            16       Australia        F15                    4        T
7            17         Belgium        F16                 

In [5]:
sales_product_forecast = pd.read_csv("Great_Outdoors_Data_SQLite/GO_SALES_PRODUCT_FORECASTData.csv")
sales_inventory_levels = pd.read_csv("Great_Outdoors_Data_SQLite/GO_SALES_INVENTORY_LEVELSData.csv")

In [6]:
# Remove tables that we not gonna use
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', '_i2', 'sql', 'warn', 'pd', '_i3', '_i4', 'conn1', 'conn2', 'conn3', 'i', 'conn', 'tables', 'table', 'age_group0', 'country0', 'retailer0', 'retailer_contact0', 'retailer_headquarters0', 'retailer_segment0', 'retailer_site0', 'retailer_type0', 'sales_demographic0', 'sales_territory0', 'country1', 'order_details1', 'order_header1', 'order_method1', 'product1', 'product_line1', 'product_type1', 'retailer_site1', 'return_reason1', 'returned_item1', 'sales_branch1', 'sales_staff1', 'SALES_TARGETData1', 'sqlite_sequence1', 'course2', 'sales_branch2', 'sales_staff2', 'satisfaction2', 'satisfaction_type2', 'training2', '_i5', 'sales_product_forecast', 'sales_inventory_levels', '_i6'])


In [11]:

# Laten we voor nu er vanuit gaan dat sales_percent de waarde is die we moeten voorspellen
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# 0    age_group
# 2    retailer
# 5    retailer_segment
# 6    retailer_site
# 7    retailer_type
# 8    sales_demographic
# 9    sales_territory

# 1    order_details
# 2    order_header
# 3    order_method
# 4    product
# 5    product_line
# 6    product_type
# 10    sales_branch
# 12    SALES_TARGETData

tables = ['age_group0', 'retailer0', 'retailer_segment0', 'retailer_site0', 'retailer_type0', 'sales_demographic0', 
          'sales_territory0', 'order_details1', 'order_header1', 'order_method1', 'product1', 'product_line1', 'product_type1', 
          'sales_branch1', 'sales_staff1', 'SALES_TARGETData1']


def merge_dataframes(tables):
    merged_df = globals()[tables[0]]
    tables = tables[1:]

    while tables:
        for i, table in enumerate(tables):
            common_columns = set(merged_df.columns).intersection(set(globals()[table].columns))
            if common_columns:
                merged_df = pd.merge(merged_df, globals()[table], how='outer', on=list(common_columns))
                tables.pop(i)
                break
        else:
            break

    return merged_df

hi = merge_dataframes(tables)
print(hi)

exit()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(mean_squared_error(y_test, y_pred))


KeyboardInterrupt: 